<a href="https://colab.research.google.com/github/YasminGarcia1210/MAESTRIA_DATOS/blob/main/Tarea_1A__Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# indicate your Spark version, here we use Spark 3.5 with pydeequ 1.4.0
os.environ["SPARK_VERSION"] = '3.5'

In [ ]:
#pip install pydeequ
#pip install sagemaker-pyspark


In [ ]:
from pyspark.sql import SparkSession, Row, DataFrame
import json
import pandas as pd
import sagemaker_pyspark

import pydeequ
classpath = ":".join(sagemaker_pyspark.classpath_jars())

spark = (SparkSession
    .builder
    .config("spark.driver.extraClassPath", classpath)
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate())

In [ ]:
data=pd.read_csv("test.csv")
data.head(3)

In [ ]:
data.describe()

In [ ]:
data.columns

In [ ]:
df = spark.read.csv("test.csv",header=True, inferSchema=True)
df.printSchema()

In [ ]:
df.show(4)

In [ ]:
from pyspark.sql.functions import col, isnan, when, count

df.select([count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in df.columns]).show()


In [ ]:
simpleAnalysisResult = AnalysisRunner(spark) \
                         .onData(df) \
                         .addAnalyzer(Size()) \
                         .addAnalyzer(Completeness("id")) \
                         .run()

simpleResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, simpleAnalysisResult)
simpleResult_df.show(truncate=False)


In [ ]:
# Añadir uno por uno y verificar
def run_analysis(analyzers):
    result = AnalysisRunner(spark).onData(df)
    for analyzer in analyzers:
        result = result.addAnalyzer(analyzer)
    return result.run()

# Analizadores en pasos
analyzers_step1 = [
    Size(),
    Completeness("id")
]

analyzers_step2 = [
    ApproxCountDistinct("ram"),
    Mean("battery_power")
]

analyzers_step3 = [
    Compliance("high ram", "ram >= 3000"),
    Compliance("4G devices", "four_g == 1")
]

analyzers_step4 = [
    Correlation("px_height", "px_width"),
    Correlation("clock_speed", "battery_power")
]

# Ejecutar y verificar cada paso
result_df1 = AnalyzerContext.successMetricsAsDataFrame(spark, run_analysis(analyzers_step1))
result_df1.show(truncate=False)

result_df2 = AnalyzerContext.successMetricsAsDataFrame(spark, run_analysis(analyzers_step2))
result_df2.show(truncate=False)

result_df3 = AnalyzerContext.successMetricsAsDataFrame(spark, run_analysis(analyzers_step3))
result_df3.show(truncate=False)

result_df4 = AnalyzerContext.successMetricsAsDataFrame(spark, run_analysis(analyzers_step4))
result_df4.show(truncate=False)


In [ ]:
from pydeequ.analyzers import *

analysisResult = AnalysisRunner(spark) \
                    .onData(df) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(Completeness("id")) \
                    .addAnalyzer(Mean("battery_power")) \
                    .addAnalyzer(Mean("clock_speed")) \
                    .addAnalyzer(Compliance("high ram", "ram >= 3000")) \
                    .addAnalyzer(Compliance("4G devices", "four_g == 1")) \
                    .addAnalyzer(Compliance("3G devices", "three_g == 1")) \
                    .addAnalyzer(Compliance("Dual SIM", "dual_sim == 1")) \
                    .addAnalyzer(Correlation("px_height", "px_width")) \
                    .addAnalyzer(Correlation("clock_speed", "battery_power")) \
                    .addAnalyzer(Correlation("int_memory", "ram")) \
                    .addAnalyzer(Correlation("mobile_wt", "battery_power")) \
                    .addAnalyzer(Correlation("talk_time", "battery_power")) \
                    .addAnalyzer(Correlation("sc_h", "sc_w")) \
                    .addAnalyzer(Correlation("fc", "pc")) \
                    .addAnalyzer(Correlation("battery_power", "ram")) \
                    .addAnalyzer(Correlation("n_cores", "clock_speed")) \
                    .run()

result_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
result_df.show(truncate=False)


In [ ]:
analysisResult_pandas_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult, pandas=True)
analysisResult_pandas_df